To demonstrate pxp callbacks (i.e., Python calling Prolog which then calls Python) we start by importing a module that contains XSB.  

However, first we have to set up the Python paths, meaning thaat we need to add to sys.path the directory containing **px**, which is in `$XSB_ROOT/XSB/packages/xsbpy/px` Assuming this notebook is run from `$XSB_ROOT/XSB/examples/python_examples` we can set the relative path as follows.  (Otherwise, add the absolute path to the `pd` directory.) For the purposes of this notebook, it's ok to add to the beginning of `sys.path`

In [ ]:
import sys
sys.path.insert(0,'../../packages/xsbpy/px')   

In [ ]:
from tpx import *

The above command installs the `tpx` module, which includes both **px** and various tests.  Also, note that **xsbpy** is also loaded on importing **px** (this is due to my `xsbrc.P` initialization file

So now let's load some Prolog that calls Python:

In [ ]:
ensure_loaded('xp_unicode') 

The file `xp_unicode` is an XSB library that uses the Python 
standard library to add a bit more unicode functionality to XSB -- 
the library is small right now, but will grow.  In any case, in the following Python call the Prolog predicate uses a callback to capitalize the Greek text.


In [ ]:
px_qdet('xp_unicode','unicode_upper','Η Google μου το μετέφρασε')  

`xp_unicode:unicode_upper/2` is defined as                                                                                        
unicode_upper(Lower,Upper):-                                              pyfunc(xp_unicode,unicode_upper(Lower),Upper).                                               
So in the above call, Python returned an answer to Prolog which returned it back to Python.    

The callbacks don't seem to require anything special -- for callbacks, XSB just calls `pyfunc/3`, `pydot/4` as it would ordinarily.  The next example uses a new test file, `testcb.P` that contains the rule:                                   
                            
test_json(Ret):-       
&emsp; pyfunc(xp_json,prolog_loads(     '"name":"Bob","languages":"English","Fench","GERMAN"]}'),Ret).

In [ ]:
ensure_loaded('testcb')

In [ ]:
px_qdet('testcb','test_json')

In this example, Python read a JSON string and created a Python dictionary.  This dictionary was translated to a Prolog pyDict/1 term when pfunc/3 returned to Prolog.  Then when the Prolog predicate succeeded, the dictionary was retranslated to Python.

As the next example, we show how a class created by a Python callback is available to the top-level Python.  They should just be the same Python -- so its just a sanity check.   The Prolog `testcb` module contains the rule:

test_class(Name,Obj):-                                                                           
    pyfunc('Person','Person'(Name,35),Obj).  
    
That creates an instance of the `Person` class and passes back the object reference.

In [ ]:
NewClass,TV = px_qdet('testcb','test_class','joe')   

In [ ]:
NewClass.name

Thus, the callback can communicate with the top Python either by passing the information explicitly through Prolog or by changing the state of the Python store.  Of course references to created objects need to be coodinated by explicit object references (as above) or some other means.
